In [2]:
from rdflib import OWL, Graph
from rdflib.namespace import RDFS
from owlready2 import get_ontology, default_world
import xmltodict
import time
from rdflib import URIRef
import random
import pandas as pd
import os, sys

# one liner to import cogtext package from `../python` folder.
#if '../python' not in sys.path: sys.path.append('../python');
from python.cogtext import *


In [5]:
owl_file = 'data/ontologies/efo.owl'
owl_prefix = 'http://www.semanticweb.org/morteza/ontologies/2019/11/executive-functions-ontology#'

ONTOLOGY = get_ontology(owl_file).load()
GRAPH = default_world.as_rdflib_graph()

def query(graph, parent_cls='Task'):
    """Function to query tasks, constructs, regions, etc.

    ## Returns
    A list of labels
    """

    cls_name = parent_cls[1:] if parent_cls.startswith(":") else parent_cls

    sparql_query = f"""
    prefix : <{owl_prefix}>

    SELECT ?label ?pubmed_query
    WHERE {{
    ?cls rdfs:subClassOf* :{cls_name};
          :pubmedQuery ?pubmed_query;
          rdfs:label ?label
    }}
    """

    print(sparql_query)
    # select the all rdfs:labels, flatten the list of labels, and convert them to python string
    labels = [labels for labels in graph.query(sparql_query)]
    pubmed_queries = {l[0].toPython(): l[1].toPython() for l in labels}
    return pubmed_queries


In [6]:
# pubmed_queries = query(GRAPH, 'BaggettaTask')   # dictionary of task_name -> pubmed_query
# print(f'{len(pubmed_queries)} tasks found in the EF ontology (baggetta2016)')

pubmed_queries = query(GRAPH, 'CognitiveProcess')   # dictionary of task_name -> pubmed_query
pubmed_queries



    prefix : <http://www.semanticweb.org/morteza/ontologies/2019/11/executive-functions-ontology#>

    SELECT ?label ?pubmed_query
    WHERE {
    ?cls rdfs:subClassOf* :CognitiveProcess;
          :pubmedQuery ?pubmed_query;
          rdfs:label ?label
    }
    


{'Focused Attention': '("Focused Attention" AND "Cognitive"[TIAB])',
 'Selective Attention': '("Selective Attention" AND "Cognitive"[TIAB])',
 'Behavioral Control': '("Behavioral Control" and "Cognitive"[TIAB])',
 'Behavioral Regulation': '("Behavioral Regulation" and "Cognitive"[TIAB])',
 'Central Executive': '("Central Executive" and "Cognitive"[TIAB])',
 'Cognitive Flexibility': '("Cognitive Flexibility"[TIAB])',
 'Cognitive Control': '("Cognitive Control"[TIAB])',
 'Context Processing': '("Context Processing" and "Cognitive"[TIAB])',
 'Core Executive Functions': '(Core Executive Functions[TIAB])',
 'Inhibition': '("Inhibition" AND "Cognitive"[TIAB])',
 'Inhibitory Control': '("Inhibitory Control" AND "Cognitive"[TIAB])',
 'Interference Control': '("Interference Control" AND "Cognitive"[TIAB])',
 'Cognitive Inhibition': '("Cognitive Inhibition"[TIAB])',
 'Response Inhibition': '("Response Inhibition" AND "Cognitive"[TIAB])',
 'Working Memory': '("Working Memory" AND "Cognitive"[TIAB

# search and cache hits

Let's search PubMed for each task query (`ef:pubmedQuery`), then cleanup the XML results into a CSV with the following columns: abstract, title, and a reference to the metadata (pmid).


In [8]:
for task_label, pubmed_query in pubmed_queries.items():
    task_label = task_label.replace('/','') 
    fname = Path('../data/pubmed/.cache') / (task_label + '.xml')

    if not fname.exists():
        search_and_store(pubmed_query, fname)


def find_mesh(mesh_list):
    """Extracts MeSH names from a list of XML MedlineCitation.MeshHeadingList.MeshHeading tags."""
    if not isinstance(mesh_list, list):
        return []

    mesh_names = [h['DescriptorName']['#text'] for h in mesh_list]# if d['DescriptorName']['@MajorTopicYN'] == 'Y']
    return mesh_names


# now cleanup and convert all abstracts into CSV files
for task_label in pubmed_queries.keys():

    task_fname = task_label.replace('/','')

    xml_file = Path('../data/pubmed/.cache') / (task_fname + '.xml')
    csv_file = Path('../data/pubmed/tests') / (task_fname + '.csv')

    if not csv_file.exists():
        with open(xml_file, 'r') as f:
            xml_content = xmltodict.parse(f.read())
            if 'PubmedArticleSet' in xml_content:
                print(f'[XML2CSV] converting "{task_label}" dataset...')

                df = pd.json_normalize(xml_content['PubmedArticleSet']['PubmedArticle'])

                # pmid, title, and abstract
                df['pmid'] = df['MedlineCitation.PMID.#text']
                df['title'] = df['MedlineCitation.Article.ArticleTitle']
                df['abstract'] = df['MedlineCitation.Article.Abstract.AbstractText'].apply(cleanup_abstract)
                
                # publication year
                df['year'] = df['MedlineCitation.Article.Journal.JournalIssue.PubDate.Year']
                df['journal_title'] = df['MedlineCitation.Article.Journal.Title']
                df['journal_iso_abbreviation'] = df['MedlineCitation.Article.Journal.ISOAbbreviation']

                # MeSh topics (some datasets do not contain MeshHeading, e.g., Spin The Pots)
                # if 'MedlineCitation.MeshHeadingList.MeshHeading' in df.columns:
                #     df['mesh'] = df['MedlineCitation.MeshHeadingList.MeshHeading'].apply(find_mesh)
                # else:
                #     df['mesh'] = None

                if 'MedlineCitation.Article.Journal.JournalIssue.PubDate.MedlineDate' in df.columns:
                    medline_year = df['MedlineCitation.Article.Journal.JournalIssue.PubDate.MedlineDate'].apply(lambda x: x[0:4] if isinstance(x, str) and len(x)>=4 else x)
                    df['year'].fillna(medline_year, inplace=True)

                # fill missing abstracts with #text value
                if 'MedlineCitation.Article.Abstract.AbstractText.#text' in df.columns:
                    df['abstract'].fillna(df['MedlineCitation.Article.Abstract.AbstractText.#text'], inplace=True)

                if 'MedlineCitation.Article.ArticleTitle.#text' in df.columns:
                    df['title'].fillna(df['MedlineCitation.Article.ArticleTitle.#text'], inplace=True)

                # workaround to discard unusual terminators in the abstracts
                df['abstract'] = df['abstract'].apply(lambda x: x.replace('\u2029', ' ') if isinstance(x, str) else x)

                df[['pmid', 'year', 'journal_title', 'journal_iso_abbreviation', 'title','abstract']].to_csv(csv_file, index=False)

print('Done!')

[PubMed] (Degraded Vigilance task* "PVT"[TIAB])
[PubMed] Succesfully stored 4 hits on NCBI history server.
[PubMed] Succesfully stored hits in ../data/pubmed/.cache/Degraded Vigilance.xml.
[PubMed] ("Delay Choice" Task*[TIAB])
[PubMed] Succesfully stored 15 hits on NCBI history server.
[PubMed] Succesfully stored hits in ../data/pubmed/.cache/Delay Choice.xml.
[PubMed] ("Dimensional Change Card" Sort*[TIAB])
[PubMed] Succesfully stored 134 hits on NCBI history server.
[PubMed] Succesfully stored hits in ../data/pubmed/.cache/DCCS (Dimentional Change Card Sort).xml.
[XML2CSV] converting "Degraded Vigilance" dataset...
[XML2CSV] converting "Delay Choice" dataset...
[XML2CSV] converting "DCCS (Dimentional Change Card Sort)" dataset...
Done!


## Data cleansing

Here, we aim to preprocess PubMed corpora and keep only those relevant metadata. Outputs of this pipeline are stored in the `data/pubmed` folder as csv files; one csv per task corpus.

The following metadata will be stored in the processed csv files:

- pmid: unique PubMed identifier of the article.
- title: escaped title in string format.
- journal_title: Journal title
- journal_iso_abbreviation: Journal ISO abbreviation
- abstract: escaped and cleanedup abstract in string format.
- year: publication year in YYYY format.
- mesh: A list of Medical Subject Headings which contains the field of research and other topics. We only keep major topics.